# **GNN_Energy.ipynb: Energy prediction for EIC Det2 KLM**
This notebook walks through the steps of loading data, training and testing a GNN for energy prediction tasks

In [1]:
# import matplotlib.pyplot as plot
import pandas as pd
import os
os.environ["DGLBACKEND"] = "pytorch"
import dgl
import torch
from dgl.data import DGLDataset
import matplotlib.pyplot as plot
import numpy as np
import torch.nn as nn
import torch
import itertools
import dgl.data
import torch.nn.functional as F
from dgl.nn import GraphConv,SumPooling,GINConv,AvgPooling,MaxPooling
from dgl.dataloading import GraphDataLoader
from tqdm import tqdm
import matplotlib.pyplot as plot
from datetime import datetime as datetime
current_date = datetime.now().strftime("%B_%d")
from torch.utils.data.sampler import SubsetRandomSampler
from scipy.spatial import ConvexHull
from GNN_util import process_df_vectorized,create_directory,HitDataset,create_fast_edge_lists,visualize_detector_graph,GIN,train_GNN,test_GNN,calculate_bin_rmse

/hpc/group/vossenlab/rck32/ML_venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


## **Load in Data**
The data is stored as pandas dataframes where each row is a hit in the detector

In [2]:
dfs = []
num_dfs = 400
for i in range(num_dfs):
    try:
        new_df = pd.read_csv(f"/hpc/group/vossenlab/rck32/eic/work_eic/macros/Timing_estimation/data/df/naive_CFD_Feb_10_50events_run_1_{i}.csv")
    except FileNotFoundError as e:
        # Skip files that failed for some reason...
        # I think these files fail due to DCC issues?
        print(f"skipping file #{i}...")
        continue
    new_df["file_idx"] = i
    dfs.append(new_df)
if(len(dfs) > 1):
    data = pd.concat(dfs)
else:
    data = dfs[0]

# Plot the data
modified_df = process_df_vectorized(data, cone_angle_deg = 40)

dataset = HitDataset(modified_df,True,connection_mode = "kNN",k = 6)
print("done")
print(len(dataset))

skipping file #35...
skipping file #56...
skipping file #60...
skipping file #83...
skipping file #92...
skipping file #111...
skipping file #121...
skipping file #144...
skipping file #158...
skipping file #180...
skipping file #194...
skipping file #204...
skipping file #293...
done
18862


In [3]:
def objective(dataset, fraction) -> tuple:

    train_frac = 0.7 * fraction
    val_frac = 0.15 * fraction
    test_frac = 0.15 * fraction
    num_train = int(np.floor(len(dataset) * train_frac))
    # num_train = 5
    num_val = int(np.floor(len(dataset) * val_frac))
    num_test = int(np.floor(len(dataset) * test_frac))
    num_examples = len(dataset)

    train_sampler = SubsetRandomSampler(torch.arange(num_train))
    val_sampler = SubsetRandomSampler(torch.arange(num_train, num_val + num_train))
    test_sampler = SubsetRandomSampler(torch.arange(num_val + num_train, num_test + num_val + num_train))

    train_dataloader = GraphDataLoader(
        dataset, sampler=train_sampler, batch_size=20, drop_last=False
    )

    val_dataloader = GraphDataLoader(
        dataset, sampler=val_sampler, batch_size=20, drop_last=False
    )

    test_dataloader = GraphDataLoader(
        dataset, sampler=test_sampler, batch_size=20, drop_last=False
    )

    run_num = 1
    geometry_type = 2


    hidden_dim = 43
    linear_capacity = 5
    early_stopping_limit = 5
    n_conv_layers = 1
    n_linear_layers = 7
    lr = 4e-4

    model = GIN(dataset.dim_nfeats,hidden_dim,dataset.dim_event_feats,n_conv_layers = n_conv_layers, n_linear_layers = n_linear_layers,linear_capacity = linear_capacity)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    model_path = f"/hpc/group/vossenlab/rck32/eic/work_eic/macros/Timing_estimation/models/{current_date}/"

    n_epochs = 300
    trained_model, train_losses, val_losses, optimizer,best_epoch = train_GNN(model,optimizer,criterion, train_dataloader, val_dataloader, n_epochs, early_stopping_limit, model_path = model_path, log_status=False)

#     plot.plot(train_losses,label = "train")
#     plot.title("Train and Val loss throughout training (log scale)")
#     plot.plot(val_losses, label = "test")
#     plot.legend()
#     plot.tight_layout()

    test_truths, test_preds, test_rmse = test_GNN(trained_model, test_dataloader)

#     plot.plot([0,5],[0,5])
#     plot.title("Test dataset results")
#     plot.scatter(test_truths,test_preds,alpha = 0.1)
#     plot.xlabel("truths")
#     plot.ylabel("preds")
#     plot.tight_layout()
    return (num_train,test_rmse)

In [4]:
options = np.linspace(0.1,1,5)
options

array([0.1  , 0.325, 0.55 , 0.775, 1.   ])

In [5]:
num_examples_list = []
rmse_list = []
for i in range(len(options)):
    num_train_examples, rmse = objective(dataset,options[i])
    num_examples_list.append(num_train_examples)
    rmse_list.append(rmse)

KeyboardInterrupt: 

In [ ]:
rmse_list = np.array(rmse_list).flatten()

In [ ]:
plot.plot(num_examples_list,rmse_list);
plot.xlabel("Number of training examples (higher is more expensive)")
plot.ylabel("Test RMSE (lower is better)")
plot.title("Test RMSE as function of dataset size")